In [1]:
import pandas as pd
import numpy as np
from scipy.special import expit

In [2]:
SOURCE_FILE_PATH = "../../data/raw/robot_churn_poc_dataset.csv"
TARGET_FILE_PATH = "../../data/curated/churn_risk_scores.csv"

In [3]:
df = pd.read_csv(SOURCE_FILE_PATH)

In [4]:
df.drop("churn_flag_next_6m", axis=1, inplace=True)

---

In [5]:
def calculate_rolling_average(
    df: pd.DataFrame,
    column: str,
    window: int,
    group_cols=("customer_id", "site_id"),
    time_col="year_month",
    min_periods=1
) -> pd.DataFrame:
    """
    Calculate rolling average for a given column grouped by customer/site.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe.
    column : str
        Column name to calculate rolling average for.
    window : int
        Rolling window size (number of months).
    group_cols : tuple
        Columns used to group the rolling calculation.
    time_col : str
        Time column (must be sortable).
    min_periods : int
        Minimum periods required for calculation.

    Returns
    -------
    pd.DataFrame
        DataFrame with a new rolling average column appended.
    """

    df = df.copy()

    # Ensure proper datetime sorting
    df[time_col] = pd.to_datetime(df[time_col])

    df = df.sort_values(list(group_cols) + [time_col])

    rolling_col_name = f"{column}_rolling_avg_{window}m"

    df[rolling_col_name] = (
        df
        .groupby(list(group_cols))[column]
        .transform(lambda x: x.rolling(window=window, min_periods=min_periods).mean())
    )

    return df

In [6]:
def calculate_trend_slope(
    df: pd.DataFrame,
    column: str,
    window: int,
    group_cols=("customer_id", "site_id"),
    time_col="year_month",
    min_periods=2
) -> pd.DataFrame:
    """
    Calculate rolling trend slope (linear regression slope) for a given column.
    The slope represents direction:
        > 0  -> increasing trend
        < 0  -> decreasing trend
        ~ 0  -> stable

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe.
    column : str
        Column to calculate slope for.
    window : int
        Rolling window size (number of months).
    group_cols : tuple
        Columns used for grouping.
    time_col : str
        Time column (must be sortable).
    min_periods : int
        Minimum number of periods required to compute slope.

    Returns
    -------
    pd.DataFrame
        DataFrame with new slope column appended.
    """

    df = df.copy()

    df[time_col] = pd.to_datetime(df[time_col])
    df = df.sort_values(list(group_cols) + [time_col])

    slope_col_name = f"{column}_slope_{window}m"

    def rolling_slope(series):
        y = series.values
        x = np.arange(len(y))

        if len(y) < min_periods:
            return np.nan

        # Linear regression slope (least squares)
        slope = np.polyfit(x, y, 1)[0]
        return slope

    df[slope_col_name] = (
        df
        .groupby(list(group_cols))[column]
        .transform(
            lambda x: x.rolling(window=window, min_periods=min_periods)
                      .apply(rolling_slope, raw=False)
        )
    )

    return df

In [7]:
def calculate_consecutive_movement(
    df: pd.DataFrame,
    column: str,
    group_cols=("customer_id", "site_id"),
    time_col="year_month"
) -> pd.DataFrame:
    """
    Calculate consecutive upward and downward movements for a numeric column.

    Creates the following columns:
        - <column>_diff
        - <column>_direction  (1=up, -1=down, 0=no change)
        - <column>_consec_up
        - <column>_consec_down

    Example:
        3 months consecutive NPS drop → consec_down = 3
    """

    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col])
    df = df.sort_values(list(group_cols) + [time_col])

    diff_col = f"{column}_diff"
    dir_col = f"{column}_direction"
    up_col = f"{column}_consec_up"
    down_col = f"{column}_consec_down"

    # Month-over-month difference
    df[diff_col] = (
        df.groupby(list(group_cols))[column]
          .diff()
    )

    # Direction: 1 up, -1 down, 0 stable
    df[dir_col] = np.sign(df[diff_col]).fillna(0)

    def compute_streak(series, direction_value):
        streak = []
        counter = 0
        for val in series:
            if val == direction_value:
                counter += 1
            else:
                counter = 0
            streak.append(counter)
        return streak

    df[up_col] = (
        df.groupby(list(group_cols))[dir_col]
          .transform(lambda x: compute_streak(x, 1))
    )

    df[down_col] = (
        df.groupby(list(group_cols))[dir_col]
          .transform(lambda x: compute_streak(x, -1))
    )

    return df

In [8]:
def calculate_behavior_deterioration_score(
    df: pd.DataFrame,
    features: list,
    window: int = 3,
    group_cols=("customer_id", "site_id"),
    time_col="year_month"
) -> pd.DataFrame:
    """
    Create unified deterioration score combining:
    - slope
    - consecutive streak
    - deviation vs rolling average

    Returns dataframe with:
        behavior_deterioration_score
    """

    df = df.copy()
    df[time_col] = pd.to_datetime(df[time_col])
    df = df.sort_values(list(group_cols) + [time_col])

    deterioration_components = []

    for col in features:

        slope_col = f"{col}_slope_{window}m"
        streak_down_col = f"{col}_consec_down"
        streak_up_col = f"{col}_consec_up"
        rolling_col = f"{col}_rolling_avg_{window}m"

        # --- Slope Component ---
        slope_component = -df[slope_col]  # negative slope = deterioration

        # --- Streak Component ---
        # For positive performance metrics → down streak is bad
        if col not in ["upgrade_failures", "lack_of_rca"]:
            streak_component = df[streak_down_col]
        else:
            # For failure metrics → up streak is bad
            streak_component = df[streak_up_col]

        # --- Rolling Deviation Component ---
        deviation = df[col] - df[rolling_col]

        if col not in ["upgrade_failures", "lack_of_rca"]:
            deviation_component = -deviation  # below avg = bad
        else:
            deviation_component = deviation   # above avg = bad

        # Normalize each component
        for comp in [slope_component, streak_component, deviation_component]:
            comp_norm = (comp - comp.mean()) / (comp.std() + 1e-6)
            deterioration_components.append(comp_norm)

    # Aggregate all components
    df["behavior_deterioration_score"] = np.mean(deterioration_components, axis=0)

    return df

In [9]:
features = [
    "usage_volume",
    "bot_utilization",
    "bot_performance",
    "associate_performance",
    "bot_uptime",
    "upgrade_failures",
    "support_sla",
    "lack_of_rca",
    "nps"
]

In [10]:
for feature in features:
    df = calculate_rolling_average(df, feature, window=3)

for feature in features:
    df = calculate_trend_slope(df, feature, window=3)

for feature in features:
    df = calculate_consecutive_movement(df, feature)

In [11]:
df = calculate_behavior_deterioration_score(
    df,
    features=features,
    window=3
)

In [12]:
# Performance deterioration (3 consecutive drops)
df["nps_3m_drop_flag"] = df["nps_consec_down"] >= 3
df["usage_3m_drop_flag"] = df["usage_volume_consec_down"] >= 3
df["utilization_3m_drop_flag"] = df["bot_utilization_consec_down"] >= 3
df["performance_3m_drop_flag"] = df["bot_performance_consec_down"] >= 3
df["uptime_2m_drop_flag"] = df["bot_uptime_consec_down"] >= 2
df["sla_2m_drop_flag"] = df["support_sla_consec_down"] >= 2

# Operational issue escalation (2+ consecutive increases)
df["failures_2m_rise_flag"] = df["upgrade_failures_consec_up"] >= 2
df["rca_2m_rise_flag"] = df["lack_of_rca_consec_up"] >= 2

In [13]:
performance_streaks = [
    "usage_volume_consec_down",
    "bot_utilization_consec_down",
    "bot_performance_consec_down",
    "associate_performance_consec_down",
    "bot_uptime_consec_down",
    "support_sla_consec_down",
    "nps_consec_down"
]

failure_streaks = [
    "upgrade_failures_consec_up",
    "lack_of_rca_consec_up"
]

df["streak_risk_score"] = (
    df[performance_streaks].sum(axis=1) +
    df[failure_streaks].sum(axis=1)
)

In [14]:
latest_df = (
    df.sort_values("year_month")
        .groupby(["customer_id", "site_id"], as_index=False)
        .tail(1)
        .copy()
)

In [15]:
score = latest_df["behavior_deterioration_score"]
latest_df["score_z"] = (score - score.mean()) / (score.std() + 1e-9)

In [16]:
alpha = 1.5
latest_df["calibrated_probability"] = expit(alpha * latest_df["score_z"])

In [17]:
def prob_to_risk(p: float) -> str:
    if p < 0.40:
        return "low"
    elif p < 0.70:
        return "medium"
    return "high"

latest_df["churn_risk"] = latest_df["calibrated_probability"].apply(prob_to_risk)

In [18]:
latest_df["calibrated_probability"] = latest_df["calibrated_probability"].map(lambda x: f"{x:.5f}")

In [19]:
predictions_df = (
    latest_df[
        [
            "customer_id",
            "site_id",
            "year_month",
            "churn_risk",
            "calibrated_probability",
            "behavior_deterioration_score",
            "usage_volume",
            "bot_utilization",
            "bot_performance",
            "bot_uptime",
            "support_sla",
            "nps",
            "usage_volume_slope_3m",
            "bot_performance_slope_3m",
            "bot_uptime_slope_3m",
            "support_sla_slope_3m",
            "nps_slope_3m",
            "nps_consec_down",
            "usage_volume_consec_down",
            "upgrade_failures_consec_up",
            "lack_of_rca_consec_up"
        ]
    ]
    .sort_values(["customer_id", "site_id"])
    .reset_index(drop=True)
)

---

In [20]:
predictions_df["churn_risk"].value_counts(normalize=True)

churn_risk
low       0.450980
high      0.294118
medium    0.254902
Name: proportion, dtype: float64

---

In [22]:
predictions_df.to_csv(TARGET_FILE_PATH, index=False)